In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras import optimizers
import keras
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

import os
import tensorflow as tf

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR) #ignore warnings

Using TensorFlow backend.


In [2]:
IMG_PATH = "data/"
print(os.listdir(IMG_PATH))

['.ipynb_checkpoints', 'validation_set', 'training_set']


In [3]:
train_path = IMG_PATH + "training_set/"
validation_path = IMG_PATH + "validation_set/"

In [4]:
def number_of_imgs(path):
    print(path)
    for value in os.listdir(path):
        print(value, "has", len(os.listdir(path + value)), "imgs")

In [5]:
number_of_imgs(train_path)
number_of_imgs(validation_path)

data/training_set/
hemmorhage_data has 80 imgs
non_hemmorhage_data has 80 imgs
data/validation_set/
hemmorhage_data has 20 imgs
non_hemmorhage_data has 20 imgs


In [6]:
img_width, img_height = 299, 299
nb_train_samples = 160
nb_validation_samples = 40
epochs = 10
batch_size = 10
checkpoint_filepath = "checkpoint/model-{epoch:02d}-{loss:.2f}.h5"

In [7]:
model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same', input_shape=(img_width, img_height, 3), activation='relu'))
model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1))
model.add(Activation('sigmoid'))
    
model.compile(
    loss='binary_crossentropy',
    optimizer=keras.optimizers.Adam(lr=0.0001),
    metrics=['accuracy'])

In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 299, 299, 32)      896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 299, 299, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 149, 149, 32)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 149, 149, 64)      18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 149, 149, 64)      36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 74, 74, 64)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 74, 74, 128)       73856     
__________

In [9]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

In [10]:
test_datagen = ImageDataGenerator(rescale=1. / 255)

In [11]:
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 160 images belonging to 2 classes.


In [12]:
validation_generator = test_datagen.flow_from_directory(
    validation_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 40 images belonging to 2 classes.


In [13]:
reduce_learning_rate = ReduceLROnPlateau(
    monitor='loss',
    factor=0.1,
    patience=5,
    min_lr=0.00001,
    verbose=1)

In [14]:
model_checkpoint = ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor="val_loss",
    save_best_only=True,
    verbose=1,
    mode="min")

In [15]:
callbacks = [reduce_learning_rate, model_checkpoint]

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size,
    callbacks=callbacks)

Epoch 1/10
16/16 [==============================] - 6s 389ms/step - loss: 0.6984 - acc: 0.4438 - val_loss: 0.6917 - val_acc: 0.5000

Epoch 00001: val_loss improved from inf to 0.69165, saving model to checkpoint/model-01-0.70.h5
Epoch 2/10
16/16 [==============================] - 2s 139ms/step - loss: 0.6922 - acc: 0.5000 - val_loss: 0.6920 - val_acc: 0.5000

Epoch 00002: val_loss did not improve from 0.69165
Epoch 3/10
16/16 [==============================] - 2s 137ms/step - loss: 0.6856 - acc: 0.5563 - val_loss: 0.6899 - val_acc: 0.5000

Epoch 00003: val_loss improved from 0.69165 to 0.68993, saving model to checkpoint/model-03-0.69.h5
Epoch 4/10
16/16 [==============================] - 2s 133ms/step - loss: 0.6892 - acc: 0.5000 - val_loss: 0.6884 - val_acc: 0.5000

Epoch 00004: val_loss improved from 0.68993 to 0.68844, saving model to checkpoint/model-04-0.69.h5
Epoch 5/10
16/16 [==============================] - 2s 132ms/step - loss: 0.6680 - acc: 0.5813 - val_loss: 0.6782 - val_a